In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
csv_path = "final.csv"
image_path = "./images"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class AllDataset(Dataset):
    def __init__(self, csv_path, image_path, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(AllDataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        self.images = []
        for Id in csv['Id']:
            Id = str(Id)
            image = Image.open(image_path+'/'+Id+'.jpg')
            self.images.append(image_transform(image))
            image.close()
        
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(csv_np[:,[2,9,10,11]], dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        return self.images[idx], self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [5]:
data_length = len(pd.read_csv(csv_path))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

image_transform = transforms.Compose([transforms.ToTensor()
                                     ])

train_dataset = AllDataset(csv_path, image_path, image_transform)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet18.fc = nn.Sequential(nn.Linear(512, 4096),
                                         nn.BatchNorm1d(4096),
                                         nn.ReLU(),
                                         nn.Dropout()
                                        )
        self.resenet18 = self.resnet18.to(device)
        
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.csvbn = nn.BatchNorm1d(80)
        self.csvfc1 = nn.Linear(220, 8192)
        self.csvfc2 = nn.Linear(8192, 8192)
        self.csvfc3 = nn.Linear(8192, 4096)
        self.csvbn1 = nn.BatchNorm1d(8192)
        self.csvbn2 = nn.BatchNorm1d(8192)
        self.csvbn3 = nn.BatchNorm1d(4096)
        
        self.fc1 = nn.Linear(8192, 8192)
        self.fc2 = nn.Linear(8192, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, 1)
        self.bn1 = nn.BatchNorm1d(8192)
        self.bn2 = nn.BatchNorm1d(4096)
        self.bn3 = nn.BatchNorm1d(2048)
        self.dropout = nn.Dropout()
    
    def forward(self, image, x, y):
        image = self.resnet18(image)
        
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.csvbn(self.fc(y))
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        
        x = self.dropout(self.act(self.csvbn1(self.csvfc1(x))))
        x = self.dropout(self.act(self.csvbn2(self.csvfc2(x))))
        x = self.dropout(self.act(self.csvbn3(self.csvfc3(x))))
        x = torch.cat((x, image), dim=1)
        
        x = self.dropout(self.act(self.bn1(self.fc1(x))))
        x = self.dropout(self.act(self.bn2(self.fc2(x))))
        x = self.dropout(self.act(self.bn3(self.fc3(x))))
        return self.fc4(x)

In [7]:
model = Net().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for Image, X1, X2, Y in train_loader:
        Image = Image.to(device)
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()
        prediction = model(Image, X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for Image_val, X1_val, X2_val, Y_val in val_loader:
            Image_val = Image_val.to(device)
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(Image_val, X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 6121.4844, val_loss: 4747.8330
[Epoch:  2] Average loss: 4465.8701, val_loss: 4863.6670
[Epoch:  3] Average loss: 4902.9707, val_loss: 4912.2871
[Epoch:  4] Average loss: 4426.3760, val_loss: 4795.3140
[Epoch:  5] Average loss: 4356.7100, val_loss: 4971.5811
[Epoch:  6] Average loss: 4519.6299, val_loss: 4766.3896
[Epoch:  7] Average loss: 4428.3555, val_loss: 4724.9756
[Epoch:  8] Average loss: 4302.3032, val_loss: 4112.8955
[Epoch:  9] Average loss: 4696.2007, val_loss: 4430.3809
[Epoch: 10] Average loss: 4293.1733, val_loss: 4669.3936
[Epoch: 11] Average loss: 4423.8672, val_loss: 4366.0967
[Epoch: 12] Average loss: 4196.5068, val_loss: 4418.1235
[Epoch: 13] Average loss: 4246.7329, val_loss: 4502.8853
[Epoch: 14] Average loss: 4181.0073, val_loss: 4518.6704
[Epoch: 15] Average loss: 4193.9897, val_loss: 4575.7549
[Epoch: 16] Average loss: 4124.6880, val_loss: 4084.5862
[Epoch: 17] Average loss: 4095.0752, val_loss: 4142.0742
[Epoch: 18] Average loss: 3982.

[Epoch: 144] Average loss: 1119.5492, val_loss: 1519.6450
[Epoch: 145] Average loss: 1229.3499, val_loss: 1233.1434
[Epoch: 146] Average loss: 1108.5319, val_loss: 1057.6321
[Epoch: 147] Average loss: 1011.5131, val_loss: 1209.4021
[Epoch: 148] Average loss: 1317.8910, val_loss: 1220.4167
[Epoch: 149] Average loss: 1059.8127, val_loss: 1171.1582
[Epoch: 150] Average loss: 1181.0359, val_loss: 979.4678
[Epoch: 151] Average loss: 1023.9987, val_loss: 1158.6132
[Epoch: 152] Average loss: 1048.9502, val_loss: 1138.7140
[Epoch: 153] Average loss: 1097.1111, val_loss: 1395.2534
[Epoch: 154] Average loss: 1107.5942, val_loss: 1876.5452
[Epoch: 155] Average loss: 1062.6178, val_loss: 1267.5857
[Epoch: 156] Average loss: 1193.1984, val_loss: 1013.6439
[Epoch: 157] Average loss: 1186.7244, val_loss: 977.4416
[Epoch: 158] Average loss: 1058.6919, val_loss: 1141.8003
[Epoch: 159] Average loss: 949.6273, val_loss: 1238.2766
[Epoch: 160] Average loss: 1061.7294, val_loss: 1171.4208
[Epoch: 161] Aver

In [ ]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for Image_test, X1_test, X2_test, Y_test in test_loader:
        Image_test =Image_test.to(device)
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(Image_test, X1_test, X2_test)
        mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")